此文件主要介绍两点：

1. Tensor和Numpy类似，但是Tensor可以在GPU上运行。
2. Tensor的计算图和自动求导。

此文件用一个三阶多项式third order polynomial 来拟合f(x) = sin(x)


# Numpy


Numpy 提供了一个 n 维数组对象，以及许多用于操作这些数组的函数。Numpy 是一个通用的科学计算框架；它对计算图、深度学习或梯度一无所知

In [1]:
import numpy as np
import math


# 创建随机的输入和输出数据
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# 随机初始化权重
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # 前向传播，计算预测值y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 计算和打印损失
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # 反向传播，计算损失对所有参数的梯度，它根据损失函数关于模型参数的梯度来更新参数
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 1178.4127779675523
199 783.5755752158345
299 522.0924202636127
399 348.9041841141641
499 234.18355713718765
599 158.18308260680638
699 107.82766690883935
799 74.45931259061932
899 52.344362347855736
999 37.68538046718792
1099 27.967028005177777
1199 21.52300517925343
1299 17.249325778645826
1399 14.414460704545318
1499 12.53361199460723
1599 11.285446748647667
1699 10.456945605702828
1799 9.90686881672276
1899 9.54155220116942
1999 9.298869683460017
Result: y = 0.006795945642563865 + 0.8363390183640453 x + -0.0011724137083658841 x^2 + -0.0904284071362911 x^3


## 更新参数

在给出的例子中，使用的是一个简单的三阶多项式模型来拟合数据，模型的形式为：

$$ y_{pred} = a + b \cdot x + c \cdot x^2 + d \cdot x^3 $$

损失函数是均方误差（Mean Squared Error, MSE），它是预测值 \(y_{pred}\) 和真实目标值 \(y\) 差的平方和，对于所有的数据点 \(i\)，可以表示为：

\[ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{pred,i} - y_i)^2 \]

其中，\(n\) 是数据点的数量。

现在，我们需要计算损失函数关于每个参数的梯度。对于参数 \(a, b, c, d\)，梯度的计算公式如下：

对 \(a\) 的梯度：

\[ \frac{\partial \text{MSE}}{\partial a} = \frac{2}{n} \sum_{i=1}^{n} (y_{pred,i} - y_i) \]

对 \(b\) 的梯度：

\[ \frac{\partial \text{MSE}}{\partial b} = \frac{2}{n} \sum_{i=1}^{n} (y_{pred,i} - y_i) \cdot x_i \]

对 \(c\) 的梯度：

\[ \frac{\partial \text{MSE}}{\partial c} = \frac{2}{n} \sum_{i=1}^{n} (y_{pred,i} - y_i) \cdot x_i^2 \]

对 \(d\) 的梯度：

\[ \frac{\partial \text{MSE}}{\partial d} = \frac{2}{n} \sum_{i=1}^{n} (y_{pred,i} - y_i) \cdot x_i^3 \]

在每次迭代中，我们可以使用这些梯度来更新模型的参数，通常使用梯度下降法。参数的更新公式为：

\[ \theta_{new} = \theta_{old} - \eta \cdot \frac{\partial \text{MSE}}{\partial \theta} \]

其中，\(\theta\) 表示模型参数（\(a, b, c, d\) 中的一个），\(\eta\) 是学习率，\(\frac{\partial \text{MSE}}{\partial \theta}\) 是损失函数关于参数 \(\theta\) 的梯度。通过这种方式，我们可以逐步调整参数，以最小化损失函数。

计算图

在PyTorch中，autograd包是一个核心组件，它提供了自动微分的功能。当你使用autograd时，神经网络的前向传播（forward pass）会构建一个计算图（computational graph）。这个计算图详细记录了数据（张量）在网络中流动的路径，以及在这个过程中所进行的一系列操作。

计算图是一种有向无环图（Directed Acyclic Graph, DAG），其中节点表示张量（即数据），边表示操作（如加法、乘法、激活函数等）。这种图形结构使得PyTorch能够追踪张量之间的依赖关系，从而在反向传播（backpropagation）过程中自动计算梯度。

当你执行一个操作，比如x = y * z，autograd会自动记录这个操作，并将x标记为y和z的函数。在反向传播过程中，PyTorch会使用这个信息来计算x相对于y和z的梯度。这是因为反向传播算法需要知道如何调整网络中的权重，以便在下一次迭代中减少损失函数的值。

nodes in the graph will be Tensors, and edges will be functions that produce output Tensors from input Tensors.


torch.no_grad()是一个非常有用的上下文管理器，它用于暂时停止所有在其作用域内的张量（tensors）的梯度计算。这通常在手动更新权重时使用，尤其是在使用预训练模型或者在模型的某些部分不希望通过自动梯度计算（autograd）来跟踪操作时。

当你创建一个张量并设置requires_grad=True时，PyTorch会跟踪对这个张量的所有操作，以便在反向传播时计算梯度。这是训练神经网络时的一个关键特性，因为它允许模型通过梯度下降等优化算法来学习。

然而，在某些情况下，你可能不希望跟踪这些操作。例如，当你在模型训练完成后进行推理，或者当你手动更新模型的权重时（而不是通过PyTorch的自动梯度计算）。在这些情况下，使用torch.no_grad()可以减少内存消耗，因为梯度计算需要额外的内存，并且可以提高计算效率，因为梯度计算是一个相对昂贵的操作。

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

How to define new autograd functions in PyTorch
In PyTorch we can easily define our own autograd operator by defining a subclass of `torch.autograd.Function` and implementing the forward and backward functions.

$$ y = a + b P_3(c+dx) $$

where $$ P_3(x) = 1/2 * (5x^3 - 3x)$$  is the Legendre polynomial of degree three.

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

"""
为什么forward只计算P3，不计算y呢？
backward中的grad_output是什么？

"""

class LegendrePolynomial3(torch.autograd.Function):
    """
    custome the autograd function
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        接受两个参数：ctx和input。ctx是一个上下文对象，用于在前向传播和后向传播之间保存信息。
        input是传入的输入数据，通常是一个张量（Tensor）。
        输出是一个张量。
        """
        ctx.save_for_backward(input)
        """
        使用ctx对象的save_for_backward方法来保存input数据。
        这是为了在计算梯度时能够访问到前向传播的输入数据。保存的数据将在后向传播时通过ctx对象提供。
        
        """
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        反向传播的核心计算。它计算了损失函数相对于输入的梯度。这里使用了链式法则，其中 grad_output 是高阶导数，input 是前向传播的输入。
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1) # 


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

"""
创建了一个张量a，其所有元素都被初始化为0.0。()表示张量的形状，这里是一个空元组，意味着创建的是一个标量（单个值的张量）。
device=device指定了张量应该在哪个设备上存储数据，dtype=dtype指定了数据类型（比如浮点数），
而requires_grad=True表示这个张量在自动梯度计算中需要跟踪所有操作，这对于训练神经网络时进行反向传播是必要的。
"""
# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        """
        在某些情况下，我们可能需要手动将这些梯度设置为零，这通常发生在以下几种情况：

        梯度累积：在某些训练算法中，比如RMSprop或Adam等自适应学习率算法，梯度累积可能会影响权重的更新。为了确保这些算法能够正确地调整学习率，我们需要在更新权重后清除梯度。

        避免梯度爆炸：在训练深度神经网络时，梯度可能会变得非常大，这被称为梯度爆炸问题。通过在每次更新权重后将梯度设置为零，我们可以防止梯度无限增长。
        """
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

PyTorch: nn

将上面的代码组织组织，将计算过程组织成层layers，因为这样可以使代码更加模块化、易于理解和复用。在PyTorch中，nn包提供了一种非常方便的方法来构建神经网络。
nn 里面的Modules是一个包含了许多层的模块，同时也包含了将输入数据映射到输出数据的forward(input)方法。   
nn 包中还包含了许多常用的损失函数，比如均方误差（Mean Squared Error, MSE）损失函数，以及优化算法，比如随机梯度下降（Stochastic Gradient Descent, SGD）等。

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# 此处的例子：y is a linear function of (x, x^2, x^3)
x = torch.linspace(-math.pi, math.pi, 2000) # x是一维张量，shape为(2000,)
y = torch.sin(x)

# x.unsqueeze(-1)表示在最后一个维度后面增加一个维度。x.unsqueeze(-1) 的shape是 (2000, 1)
# p的 shape是 (3,), 
# .pow(p) 操作将使用广播语义（broadcasting semantics）来执行
# 广播机制会自动将 p “扩展” 以匹配 x.unsqueeze(-1) 的形状，从而创建一个形状为 [2000, 3] 的新张量。
# 在这个过程中，p 中的每个元素会分别与 x.unsqueeze(-1) 中的每个元素进行幂运算。


p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)  # 对于 x 中的每个元素，xx 中的对应元素将是 [x^1, x^2, x^3]


# 用nn package定义模型为一系列层。nn.Sequential是一个包含其他模块的模块，它按顺序应用这些模块以产生输出。
# Linear Module使用线性函数从输入中计算输出，并保存其权重和偏置的内部张量。它接受一个形状为 [batch_size, 3] 的输入张量，并输出一个形状为 [batch_size, 1] 的张量。
# 在这个层中，每个输入样本的 3 个特征会被映射到一个输出值。这个层有一个权重矩阵，其形状为 [1, 3]（因为输出维度是 1），和一个偏置向量，其形状为 [1]。

# Flatten层将线性层的输出展平为1D张量，以匹配y的形状。PyTorch 从输入张量的第一个维度（0）开始，直到第二个维度（1）结束，将这两者之间的所有维度合并为一个维度。
# 当输入张量为(batch_size,3)时，输出张量的形状为(batch_size,1)。



model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)



# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum') # reductio的取值：'sum'：计算的是总损失。'mean': 默认值，计算所有样本损失的平均值。'none': 不进行约简，返回每个样本的损失。

learning_rate = 1e-6
for t in range(2000):

    # 步骤一： 通过将 x 传递给模型来计算预测的 y。
    y_pred = model(xx)

    # 步骤二：计算loss
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 步骤三：将梯度归零，反向传播
    model.zero_grad()
    loss.backward()

    # 步骤四：更新权重
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

PyTorch: optim

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)


for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')